In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
import datetime
import json
import glob
import sys
import os
from itertools import combinations
from collections import Counter

import openpyxl

import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.grid'] = False

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

pd.options.display.float_format = '{:.2f}'.format

In [5]:
class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
        print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """
        
        

        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            df = pd.DataFrame(result['results'][0]['data'])[['period']]
            for i in range(len(self.keywordGroups)):
                tmp = pd.DataFrame(result['results'][i]['data'])
                tmp = tmp.rename(columns={'ratio': result['results'][i]['title']})

                df = pd.merge(df, tmp, how='left', on=['period'])

            self.df = df.rename(columns={'period': '날짜'})
            self.df['날짜'] = pd.to_datetime(self.df['날짜'])
            
        else:
            print("Error Code:" + rescode)
            
        return self.df


In [6]:
Kospi_df=pd.read_excel("13.xlsx",header=None).T
Kospi_list=Kospi_df.loc[0,:]
Kospi_Data=Kospi_df.loc[1:,:]

In [7]:
client_id = "GjUxJUOHwe6yND6FaSlB"
client_secret = "qKQ6UJToQB"

# 요청 파라미터 설정
startDate = "2022-01-01"
endDate = "2022-12-26"
timeUnit = 'date'
device = ''
ages = []
gender = ''

In [8]:
Number=0
Jusik_list=[]
Kospi=Kospi_list
#Kospi_Data
for i in range(40):
    Count_list=Kospi[Number:Number+5]
    keyword_group_set={}
    for j in range(5):
        keyword_group_set['keyword_group_'+str(j+1)]={'groupName':Count_list[Number+j],'keywords':[str(x) for x in Kospi_Data.loc[:19,Number+j].dropna()]+[Count_list[Number+j]]}
        for p in keyword_group_set['keyword_group_'+str(j+1)]['keywords']:
            if str(p) =='NaN':
                keyword_group_set['keyword_group_'+str(j+1)]['keywords'].remove('nan') 
    Number=Number+5 
    naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)
    for j in range(5):
        naver.add_keyword_groups(keyword_group_set['keyword_group_'+str(j+1)])
    df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)
    Name_list =df[df==100].isnull().sum()
    Name_list=Name_list[1:6]
    Jusik_Name=Name_list.idxmin() #100을 유지하는 값.
    Jusik_list.append(Jusik_Name)
    
    
Number=0
Jusik_list2=[]
Jusik_list_np=np.array(Jusik_list)
Kospi_list_np=np.array(Kospi_list)
number_np=[]
for i in Jusik_list_np:
    a=np.where(Kospi_list_np==i)
    a=int(a[0])
    number_np.append(a)
    
Kospi_Data1=Kospi_Data[number_np]
Kospi_Data1.columns=range(40)

    
for i in range(8):
    Count_list=Jusik_list[Number:Number+5]
    keyword_group_set={}
    for j in range(5):
        keyword_group_set['keyword_group_'+str(j+1)]={'groupName':Count_list[j],'keywords':[str(x) for x in Kospi_Data1.loc[:19,Number+j].dropna()]+[Count_list[j]]}
        for p in keyword_group_set['keyword_group_'+str(j+1)]['keywords']:
            if str(p) =='NaN':
                keyword_group_set['keyword_group_'+str(j+1)]['keywords'].remove('nan') 
        
    Number=Number+5 
    naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)
    for j in range(5):
        naver.add_keyword_groups(keyword_group_set['keyword_group_'+str(j+1)])
    df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)
    Name_list =df[df==100].isnull().sum()
    Name_list=Name_list[1:6]
    Jusik_Name=Name_list.idxmin() #100을 유지하는 값.
    Jusik_list2.append(Jusik_Name)
    
    

    
    
    
    
Jusik_list3=list(combinations(Jusik_list2, 5))
Jusik_list4=[]
Empty=[]
Number=0


for Count_list in Jusik_list3: 
    Jusik_list_np=np.array(Count_list)
    Kospi_list_np=np.array(Kospi_list)
    number_np=[]
    for i in Jusik_list_np:
        a=np.where(Kospi_list_np==i)
        a=int(a[0])
        number_np.append(a)
    
    Kospi_Data1=Kospi_Data[number_np]
    Kospi_Data1.columns=range(5)
    keyword_group_set={}
    for j in range(5):
        keyword_group_set['keyword_group_'+str(j+1)]={'groupName':Count_list[j],'keywords':[str(x) for x in Kospi_Data1.loc[:19,j].dropna()]+[Count_list[j]]}
        for p in keyword_group_set['keyword_group_'+str(j+1)]['keywords']:
            if str(p) =='NaN':
                keyword_group_set['keyword_group_'+str(j+1)]['keywords'].remove('nan') 
    Number=Number+5 
    naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)
    for j in range(5):
        naver.add_keyword_groups(keyword_group_set['keyword_group_'+str(j+1)])
    df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)
    Name_list =df[df==100].isnull().sum()
    Name_list=Name_list[1:6]
    Jusik_Name=Name_list.idxmin() #100을 유지하는 값.
    Jusik_list4.append(Jusik_Name)

print(Jusik_list4)

counter = Counter(Jusik_list4)
print(dict(counter))


counter=dict(counter)
for i in list(counter.keys()):
    if counter[i]==35:
        counter['1등']=i

print(counter)  



>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGr

>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGr

In [18]:
Data=pd.DataFrame()
First=counter['1등']
Number=0
Jusik_list5=[]

Kospi_list1=Kospi_list.copy()
Kospi_Data1=Kospi_Data.copy()
First_number=np.where(First == Kospi_list1)
First_number=int(First_number[0])
Kospi_Data1.drop(labels=First_number, axis=1,inplace=True)
del Kospi_list1[First_number]
Kospi_list1.index=range(199)
Kospi_Data1.columns=range(199)
for i in range(49):
    Count_list=Kospi_Data1[Number:Number+4]
    keyword_group_set={}
    for j in range(4):
        keyword_group_set['keyword_group_'+str(j+1)]={'groupName':Kospi_list1[Number+j],'keywords':[str(x) for x in Kospi_Data1.loc[:19,Number+j].dropna()]+[Kospi_list1[Number+j]]}
        for p in keyword_group_set['keyword_group_'+str(j+1)]['keywords']:
            if str(p) =='NaN':
                keyword_group_set['keyword_group_'+str(j+1)]['keywords'].remove('nan')         
    keyword_group_set['keyword_group_5']={'groupName':First,'keywords':[x for x in Kospi_Data[First_number]]}
    Number=Number+4 
    
    
    
    
    naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)
    for j in range(5):
        naver.add_keyword_groups(keyword_group_set['keyword_group_'+str(j+1)])
    df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

    for i in df.columns[1:5]:
        Data[i]=df[i]

>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5
>>> Num of keywordGr

In [19]:
Count_list=Kospi_list.loc[Number:Number+3]
Count_list.index=range(4)
keyword_group_set={}
for j in range(3):
    keyword_group_set['keyword_group_'+str(j+1)]={'groupName':Count_list[j],'keywords':[Count_list[j]]}
    keyword_group_set['keyword_group_4']={'groupName':First,'keywords':[First]} 
naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)
for j in range(4):
    naver.add_keyword_groups(keyword_group_set['keyword_group_'+str(j+1)])
print(keyword_group_set)
df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

for i in df.columns[1:5]:
     Data[i]=df[i]

Data1=Data.copy()

for i in Data.columns:
    Data.loc['전체 합',i]=Data[i].sum()
    Data.loc['평균',i]=Data[i].mean()

>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
{'keyword_group_1': {'groupName': '화승엔터프라이즈', 'keywords': ['화승엔터프라이즈']}, 'keyword_group_4': {'groupName': 'LG에너지솔루션', 'keywords': ['LG에너지솔루션']}, 'keyword_group_2': {'groupName': '쿠쿠홈시스', 'keywords': ['쿠쿠홈시스']}, 'keyword_group_3': {'groupName': 'HDC현대산업개발', 'keywords': ['HDC현대산업개발']}}


In [20]:
Data_All_rank=Data.loc['전체 합',:]
Data_All_rank=Data_All_rank.sort_values(ascending=False)
Data_mean_rank=Data.loc['평균',:]
Data_mean_rank=Data_mean_rank.sort_values(ascending=False)


All_Rank_data={}
number=1
for i in Data_All_rank.index:
  All_Rank_data[i]=number
  number=number+1

Mean_Rank_data={}
number=1
for i in Data_mean_rank.index:
  Mean_Rank_data[i]=number
  number=number+1


Data_Rank_df=pd.DataFrame()
Data_Rank_df['전체 합']=None
Data_Rank_df['전체 합 등수']=None
Data_Rank_df['평균']=None
Data_Rank_df['평균 등수']=None

for i in Data.columns:
  Data_Rank_df.loc[i,'전체 합']= Data_All_rank[i]
  Data_Rank_df.loc[i,'평균']= Data_mean_rank[i]


for i in Data.columns:
  Data_Rank_df.loc[i,'전체 합 등수']=All_Rank_data[i]
  Data_Rank_df.loc[i,'평균 등수']=Mean_Rank_data[i]

In [21]:
Data

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,현대차,NAVER,기아,POSCO홀딩스,셀트리온,카카오,삼성물산,KB금융,현대모비스,신한지주,LG전자,삼성생명,SK이노베이션,SK,한국전력,포스코케미칼,KT&G,하나금융지주,LG,카카오뱅크,고려아연,LG생활건강,SK텔레콤,현대중공업,HMM,삼성화재,삼성전기,S-Oil,삼성에스디에스,두산에너빌리티,엔씨소프트,KT,우리금융지주,대한항공,한화솔루션,크래프톤,기업은행,아모레퍼시픽,카카오페이,하이브,LG이노텍,현대글로비스,롯데케미칼,CJ제일제당,SK바이오사이언스,SK바이오팜,메리츠화재,메리츠금융지주,F&F,한국조선해양,HD현대,강원랜드,LG유플러스,SK스퀘어,오리온,한국항공우주,DB손해보험,삼성중공업,LG디스플레이,한온시스템,유한양행,현대제철,넷마블,삼성엔지니어링,GS,코웨이,한국타이어앤테크놀로지,금호석유,SK아이이테크놀로지,미래에셋증권,삼성카드,현대건설,한미약품,메리츠증권,SKC,한화에어로스페이스,BGF리테일,두산밥캣,한국가스공사,롯데지주,에스디바이오센서,현대미포조선,NH투자증권,호텔신라,한국금융지주,팬오션,삼성증권,쌍용C&E,포스코인터내셔널,GS리테일,아모레G,이마트,씨에스윈드,현대로템,제일기획,현대해상,동원산업,한진칼,롯데쇼핑,CJ,한미사이언스,BNK금융지주,일진머티리얼즈,키움증권,에스원,한화생명,LS,CJ대한통운,한화,신세계,농심,OCI,한솔케미칼,두산퓨얼셀,대우조선해양,한전기술,휠라홀딩스,영원무역,HL만도,한화시스템,동서,하이트진로,대한전선,LIG넥스원,GS건설,오뚜기,KCC,DB하이텍,대웅제약,대우건설,LS ELECTRIC,LX인터내셔널,효성,롯데칠성,효성첨단소재,한전KPS,효성티앤씨,녹십자,두산,롯데정밀화학,현대위아,DL이앤씨,현대백화점,SK케미칼,현대두산인프라코어,DL,동원시스템즈,한국앤컴퍼니,영풍,동국제강,대웅,에스엘,대한유화,한샘,롯데제과,신풍제약,후성,GKL,코오롱인더,종근당,일진하이솔루스,현대엘리베이,SK네트웍스,오리온홀딩스,아이에스동서,아시아나항공,신세계인터내셔날,PI첨단소재,하나투어,TKG휴켐스,풍산,한국콜마,롯데관광개발,대상,태광산업,녹십자홀딩스,한올바이오파마,한일시멘트,KG스틸,금호타이어,더블유게임즈,코스맥스,명신산업,보령,세방전지,현대그린푸드,지누스,한섬,케이카,한세실업,화승엔터프라이즈,쿠쿠홈시스,HDC현대산업개발,LG에너지솔루션
0,11.26,0.60,2.51,0.77,1.15,2.44,12.64,3.11,0.00,2.82,3.73,0.43,1.36,1.44,16.39,4.35,1.79,1.66,1.40,2.40,1.16,1.40,8.31,1.31,2.71,0.40,0.52,3.26,1.21,2.59,2.10,1.20,0.37,0.46,NaN,2.29,4.49,9.49,5.84,1.24,2.91,4.76,1.50,2.14,1.75,0.80,0.28,0.36,0.49,5.24,0.73,1.05,0.28,0.30,0.44,0.03,1.47,2.75,2.04,0.34,0.81,1.72,1.32,3.04,0.35,0.82,0.90,2.91,0.64,0.70,0.71,0.90,1.03,0.64,1.64,5.27,0.87,0.74,0.18,0.45,0.66,0.12,0.11,0.63,0.19,1.15,0.39,0.73,0.48,0.15,0.43,1.32,0.24,0.44,0.42,0.10,35.94,0.37,1.04,0.13,2.19,0.04,0.10,0.45,0.94,0.15,0.13,0.76,1.91,0.26,0.75,0.19,6.38,0.46,4.74,0.46,0.68,0.22,0.78,0.67,0.85,0.17,0.10,NaN,0.56,0.31,0.30,1.00,1.03,0.32,0.64,0.34,0.72,0.64,0.76,0.18,0.33,0.33,0.15,0.68,0.36,0.38,0.73,0.25,0.26,0.28,0.52,9.03,0.57,0.43,0.19,0.09,0.25,0.13,0.52,0.89,0.12,0.23,2.45,0.16,2.42,0.64,0.14,0.60,0.65,0.59,0.26,0.16,0.07,0.18,3.89,0.27,0.25,1.51,NaN,0.28,0.34,0.23,7.32,0.12,0.07,0.21,0.23,NaN,0.91,0.21,0.34,0.68,0.41,0.20,0.21,0.44,2.62,3.97,0.21,0.00,0.01,0.02,0.38
1,11.31,0.53,2.34,0.87,1.45,2.63,12.67,3.49,0.00,6.28,4.37,0.44,1.51,1.62,17.50,5.11,1.67,2.29,1.26,2.75,1.17,1.46,9.23,1.46,2.91,0.44,0.64,3.22,1.30,3.20,2.09,1.39,0.37,0.59,NaN,2.15,3.87,10.80,6.57,1.51,1.96,5.04,1.58,2.12,1.70,0.98,0.32,0.43,0.57,5.78,0.78,1.11,0.30,0.33,0.57,0.03,1.42,2.57,1.70,0.34,0.92,1.74,1.45,3.37,0.43,0.92,0.98,2.71,0.74,0.73,0.86,0.86,1.16,0.85,1.96,5.68,1.28,0.91,0.21,0.51,0.78,0.15,0.13,0.58,3.20,1.18,0.46,0.82,0.51,0.18,0.47,1.47,0.24,0.47,0.47,0.11,20.12,0.44,1.33,0.18,2.24,0.05,0.10,0.51,0.94,0.18,0.13,0.76,2.36,0.28,0.86,0.20,7.41,0.54,2.63,0.44,0.78,0.22,0.93,0.72,1.94,0.20,0.11,NaN,0.77,0.30,0.33,1.15,0.98,0.41,0.69,0.46,0.92,0.68,1.00,0.24,0.45,0.34,0.18,0.68,0.46,0.42,0.86,0.32,0.25,0.33,1.07,4.85,0.69,0.50,0.21,0.11,0.22,0.12,0.50,0.95,0.15,0.24,2.93,0.17,2.67,0.77,0.17,0.57,0.67,0.64,0.31,0.19,0.08,0.17,4.40,0.24,0.31,1.88,NaN,0.31,0.34,0.27,1.46,0.13,0.09,0.24,0.25,NaN,1.00,0.27,0.37,0.77,0.29,0.20,0.23,0.54,2.42,4.33,0.21,0.00,0.01,0.03,0.60
2,59.31,2.42,12.14,3.33,6.40,6.60,22.67,5.58,0.00,3.54,3.28,0.37,4.96,5.15,60.32,11.95,5.61,11.83,3.67,13.44,6.11,4.67,35.09,2.66,12.63,1.51,1.73,9.44,6.15,14.53,6.60,7.10,1.21,2.15,NaN,9.10,10.26,45.71,17.09,6.83,3.90,25.54,3.56,7.34,3.98,2.89,1.22,2.04,1.56,23.14,4.27,4.85,1.52,1.02,3.06,0.05,3.05,6.07,6.63,0.80,4.18,6.25,9.17,12.64,1.65,3.34,4.40,6.47,3.98,1.37,2.13,1.71,5.05,4.52,6.35,17.48,5.71,4.42,0.80,2.74,4.43,0.33,0.47,2.11,0.89,4.41,2.51,3.40,2.25,1.00,2.73,5.98,0.96,2.51,1.38,0.73,10.85,2.11,4.56,0.51,7.39,0.13,0.62,1.25,2.38,1.47,0.75,4.09,7.12,1.12,4.34,0.62,25.81,1.70,2.74,1.01,2.88,0.86,4.82,3.21,7.76,0.82,0.28,NaN,4.32,0.84,1.22,6.86,3.38,1.61,1.10,1.50,7.25,2.45,3.80,0.72,1.23,0.90,0.66,5.99,1.79,2.41,4.65,1.09,1.52,1.62,2.24,4.28,2.53,2.11,0.61,0.54,0.44,0.31,2.74,3.56,0.53,1.25,3.83,0.28,11.34,3.86,0.54,2.46,2.09,3.15,1.35,0.75,0.

In [22]:
Data_Rank_df

,전체 합,전체 합 등수,평균,평균 등수
삼성전자,14039.50,1,77.78,1
삼성바이오로직스,536.80,109,2.97,109
SK하이닉스,2139.04,26,11.85,26
LG화학,554.30,103,3.07,104
삼성SDI,1814.05,34,10.05,34
현대차,1997.67,27,11.07,27
NAVER,7728.85,5,42.82,5
기아,1874.55,30,10.39,30
POSCO홀딩스,19.48,196,0.13,197
셀트리온,539.77,108,3.68,92


In [17]:
import openpyxl


with pd.ExcelWriter('최종본.xlsx') as writer:
    Data.to_excel(writer, sheet_name='sheet1')
    Data_Rank_df.to_excel(writer, sheet_name='sheet2')
    Data_All_rank.to_excel(writer, sheet_name='sheet3')
    Data_mean_rank.to_excel(writer, sheet_name='sheet4')